### Virtual Distillation

This notebook provides an implementation of virtual distillation error mitigation, as described by: https://arxiv.org/pdf/2011.07064

Virtual distillation is an error mitigation which can leverage M copies of a state $\rho$ to surpress the error term. Virtual distillation describes the approximation of the error-free expectation value of an operator $O$ as:

$$<O>_{corrected} = \dfrac{Tr(O\rho^M)}{Tr(\rho^M)}$$

As described in the paper, we make use of the following equality:
$$Tr(O\rho^M) = Tr(O^{\textbf{i}}S^{(M)}\rho^{\otimes M})$$

This equation allows us to not calculate $\rho^M$, but instead use $M$ copies of $\rho$.
Hence we can implement the pseudocode as seen in algorithm 1 in the paper. Note that this notebook only provides an implementation for $M=2$

In [75]:
import cirq
import mitiq
import numpy as np

M = 2

Z = np.array([[1, 0], [0, -1]])
I = np.eye(2)
SWAP = np.array([[1, 0, 0, 0], 
                 [0, 0, 1, 0], 
                 [0, 1, 0, 0], 
                 [0, 0, 0, 1]])


## Bell state
The following code provides a bell state circuit as our $\rho$. However any N-qubit circuit works here. The circuits are in cirq package format

In [4]:
def bell_state():
    '''
    This function returns a circuit that prepares a Bell state in cirq circuit format.
    '''

    circuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(3)
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))

    return circuit

## M copies of $\rho$
If no copies of $\rho$ are provided, the following function can be used.

In [5]:
def M_copies_of_rho(rho: cirq.Circuit, M: int=2):
    '''
    Given a circuit rho that acts on N qubits, this function returns a circuit that copies rho M times in parallel.
    This means the resulting circuit has N * M qubits.
    '''
    
    # if M <= 1:
    #     print("warning: M_copies_of_rho is not needed for M <= 1")
    #     return rho

    N = len(rho.all_qubits())

    circuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(N*M)

    for i in range(M):
        circuit += rho.transform_qubits(lambda q: qubits[q.x + N*i])

    return circuit

In [6]:
# Test the M_copies_of_rho function
circuit = bell_state()
print(M_copies_of_rho(circuit, 3))

0: ───H───@───────────────────
          │
1: ───────X───────────────────

2: ───────────H───@───────────
                  │
3: ───────────────X───────────

4: ───────────────────H───@───
                          │
5: ───────────────────────X───


## Applying swaps
A copy in this context is the specific copy of $\rho$ out of the M copies in the entire circuit. \
As can be seen in the paper, we need to allow easy access to coupling qubit n of copy 1 with qubit n of copy 2 for any n $\in$ [1,N] \
This access is done by performing a series of SWAP operations such that this pattern results in the circuit where qubit n of copy 1 is stacked above qubit n of copy 2.\
The SWAPs are returned as a list of tuples which store the indices of the qubits that have to be swapped in order. 

In [7]:
# This algorithm only works for M = 2
def generate_swaps(l: list) -> list[tuple]:

    if len(l) % 2 != 0:
        raise ValueError("The list must have an even number of elements, since M=2")

    N = len(l) // 2

    if sorted(l) != list(range(0,2*N)):
        raise ValueError("The list must contain all the integers from 0 to 2*N-1")

    correct_list = []
    for i in range(N):
        correct_list.append(i)
        correct_list.append(i+N)

    swaps = []
    for index, value in enumerate(correct_list):
        if l[index] != value:
            l_index = l.index(value)
            l[index], l[l_index] = l[l_index], l[index]
            swaps.append((index, l_index))


    return swaps

# applies swaps to check if the generate swaps algorithm works
def apply_swaps(swaps_list: list[tuple], list_to_permute: list[int]) -> list[int]:

    permuted_list = list_to_permute.copy()
    for swap in swaps_list:
        permuted_list[swap[0]], permuted_list[swap[1]] = permuted_list[swap[1]], permuted_list[swap[0]]

    return permuted_list

In [8]:
# Testing the function
# [0,1,2,3,4,5] should map to [0,3,1,4,2,5]
# [0,1,2,3,4,5,6,7] should map to [0,4,1,5,2,6,3,7]
swaps_1 = generate_swaps([0,1,2,3])
swaps_2 = generate_swaps([0,1,2,3,4,5,6,7])

print(swaps_1)
print(apply_swaps(swaps_1, [0,1,2,3]))
print(apply_swaps(swaps_2, [0,1,2,3,4,5,6,7]))

[(1, 2)]
[0, 2, 1, 3]
[0, 4, 1, 5, 2, 6, 3, 7]


## B_i matrix generation
The B matrix as seen in the paper is defined by using the diagonalization of the $S^{(2)}_i$ operator and the observable $O^{(2)}$. The result of this code is not equal to the paper. Note how it says that some elements have freedom of phase. Thats why the B matrices can be different.

In [ ]:
def commutator(mat1, mat2):
    
    comm = mat1 @ mat2 - mat2 @ mat1
    return np.allclose(mat1 @ mat2, mat2 @ mat1), comm

def dagger(mat):
    return np.conjugate(mat).T

# working (?) for 1 qubit systems with M = 2
# TODO test
# equations are taken directly from the paper

def generate_B_i(O, S_up2):

    O_up2 = 1/2 * (np.kron(O, I) + np.kron(I, O)) # eq (7)

    _, B_i = np.linalg.eig(S_up2)

    # swap columns of B_i to match the order of the eigenvectors of A
    # TODO i dont know why this is needed, and it seems like a random permutation
    # but i assume the eq 12 and eq 13 statements are correct 
    B_i = B_i[:, [2, 0, 1, 3]]

    # eq (9)
    if not commutator(O_up2, S_up2)[0]:
        print("Violates equation 9 (commutator)")
        return None

    # eq (12)
    if not np.allclose(
        B_i @ S_up2 @ dagger(B_i),
        1/2 * (np.eye(4) + np.kron(O,I) - np.kron(I,O) + np.kron(O, O) )):
        
        print("Violates equation 12")
        return None

    # eq (13)
    if not np.allclose(
        B_i @ O_up2 @ S_up2 @ dagger(B_i), 
                    O_up2):
        print("Violates equation 13")
        return None
    
    return B_i

print(generate_B_i(Z, SWAP))

[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]


In [95]:
### Testing generate_B_i
# TODO

## The algorithm
With everything prepared, we can apply the algorithm. \
This example uses the bell state as $\rho$. As operator $O$, the pauli Z is chosen. \
Every operator has a $B_i$ unitary that has to be applied, as can be seen in the paper, we define $B_i$ for the pauli Z operator

In [89]:
def vd(rho: cirq.Circuit, M: int=2, K: int=100):
    
    # print(f"We run {K} reps which means we need M*K = {M*K} copies of rho")

    # let the circuit be 2 copies of bell state
    N = len(rho.all_qubits())
    rho = M_copies_of_rho(rho, M)

    # Bi corresponding to unitary operator O, which in this case is pauli Z
    Bi_gate = np.array([
            [1, 0, 0, 0],
            [0, np.sqrt(2)/2, -np.sqrt(2)/2, 0],
            [0, np.sqrt(2)/2, np.sqrt(2)/2, 0],
            [0, 0, 0, 1]
        ])


    Bi_gate = generate_B_i(Z, SWAP)


    Ei = [0 for _ in range(N)]
    D = 0
        
    for _ in range(K):
        
        circuit = rho.copy()


        # 1) apply swaps
        swaps = generate_swaps(list(range(2*N)))
        for swap in swaps:
            circuit.append(cirq.SWAP(cirq.LineQubit(swap[0]), cirq.LineQubit(swap[1])))


        # 2) apply Bi^(2)
        unitary = Bi_gate
        B_gate = cirq.MatrixGate(unitary)
        for i in range(0,N+1,2):
            circuit.append(B_gate(cirq.LineQubit(i), cirq.LineQubit(i+1)))

        
        # 3) apply measurements
        for i in range(M*N):
            circuit.append(cirq.measure(cirq.LineQubit(i), key=f"{i}"))
        
        
    

        # run the circuit
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)
        
        # print(circuit)

        # post processing measurements
        z1 = []
        z2 = []
        

        for i in range(2*N):
            if i % 2 == 0:
                z1.append(np.squeeze(result.records[str(i)]))
            else:
                z2.append(np.squeeze(result.records[str(i)]))

        # this one is for the pauli Z obvservable
        def map_to_eigenvalues(measurement):
            if measurement == 0:
                return 1
            else:
                return -1
            
        z1 = [map_to_eigenvalues(i) for i in z1]
        z2 = [map_to_eigenvalues(i) for i in z2]
        
        # print(z1)
        # print(z2)

        for i in range(N):
            
            productE = 1
            for j in range(N):
                if i != j:
                    productE *= ( 1 + z1[j] - z2[j] + z1[j]*z2[j] )

            Ei[i] += 1/2**N * (z1[i] + z2[i]) * productE

        productD = 1
        for j in range(N):
            productD *= ( 1 + z1[j] - z2[j] + z1[j]*z2[j] )

        D += 1/2**N * productD 
        
    Z_i_corrected = [Ei[i] / D for i in range(N)]
    # print('Z_i_corrected: ', Z_i_corrected)

    return Z_i_corrected


In [90]:
rho = bell_state()
# rho = cirq.Circuit()
# qubits = cirq.LineQubit.range(2)
# rho.append(cirq.X(qubits[0]))

# print(rho)
results = vd(rho, M, 10000)
print(results)

[0.0031, 0.0031]


### Noise simulation
Here we run the circuit with simulated depolarizing noise for a one qubit circuit that is the identity, no operations are applied on this one qubit circuit

In [91]:
# This function is used to calculate the expectation value of the Z operator on a one qubit circuit
def execute(circuit):
    """Returns Tr[ρ Z] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    
    # density matrix
    dm = cirq.DensityMatrixSimulator().simulate(circuit).final_density_matrix 
    
    # print(dm)
    return dm[0, 0].real - dm[1, 1].real

In [92]:
# Identity circuit
rho = cirq.Circuit()
qubits = cirq.LineQubit.range(1)
rho.append(cirq.I(qubits[0])) # apply identity

noise_level = 0.1
noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

true_value = execute(rho) # Fault tolerant quantum computer
noisy_value = execute(noisy_rho) # Noisy quantum computer
vd_value = vd(noisy_rho, 2, 1000)[0] # Noisy quantum computer + virtual distillation

print(f"True value: {true_value}")
print(f"Noisy value: {noisy_value}")
print(f"Virtual distillation value: {vd_value}")
print(f"Error w/o  virtual distillation: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w virtual distillation:    {abs((true_value - vd_value) / true_value):.3f}")

True value: 1.0
Noisy value: 0.8666667342185974
Virtual distillation value: 0.994279176201373
Error w/o  virtual distillation: 0.133
Error w virtual distillation:    0.006


## Randomized benchmarking circuits
### 1 qubit circuits
As an example, here we run the algorithm with the pauli Z observable on random 1 qubit benchmarking circuits. These circuits are equal in function to identity. Which means all expectation values will be 1. We see that the Virtual Distillation results of noisy circuits are more accurate. 

I tested here that VD breaks with noise level > 0.8 using the B matrix from the paper \
My B found with the generate_B_i function seems to improve on the noisy circuit until noise level > 3.5

In [97]:
# returns a random benchmarking qubit circuit equal to identity
def random_circuit(n_qubits: int, num_cliffords: int) -> list:
    return mitiq.benchmarks.randomized_benchmarking.generate_rb_circuits(n_qubits=n_qubits, num_cliffords=num_cliffords, trials=1, return_type=None, seed=None)

for i in range(10):
    rho = random_circuit(1, 10)[0]
    
    noise_level = 0.01 * i * 5
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

    true_value = execute(rho) # Fault tolerant quantum computer
    noisy_value = execute(noisy_rho) # Noisy quantum computer
    vd_value = vd(noisy_rho, 2, 1000)[0] # Noisy quantum computer + virtual distillation

    print(f"circuit: {i}, noise level: {noise_level}")
    print(f"True value: {true_value}")
    print(f"Noisy value: {noisy_value}")
    print(f"Virtual distillation value: {vd_value}")
    print()


circuit: 0, noise level: 0.0
True value: 1.0
Noisy value: 1.0
Virtual distillation value: 1.0

circuit: 1, noise level: 0.05
True value: 1.0
Noisy value: 0.20457163453102112
Virtual distillation value: 0.40588235294117647

circuit: 2, noise level: 0.1
True value: 1.0
Noisy value: 0.03224411606788635
Virtual distillation value: 0.08171206225680934

circuit: 3, noise level: 0.15
True value: 0.9999999403953552
Noisy value: 0.00922343134880066
Virtual distillation value: -0.03409090909090909

circuit: 4, noise level: 0.2
True value: 1.0
Noisy value: 0.0014835894107818604
Virtual distillation value: -0.04430379746835443

circuit: 5, noise level: 0.25
True value: 1.0
Noisy value: 3.9637088775634766e-05
Virtual distillation value: 0.013157894736842105

circuit: 6, noise level: 0.3
True value: 1.0
Noisy value: 1.3202428817749023e-05
Virtual distillation value: 0.07383966244725738

circuit: 7, noise level: 0.35000000000000003
True value: 1.0
Noisy value: 1.8775463104248047e-06
Virtual distillat

### 2 qubit circuits
here we see VD is not more accurate when noise level > 0.6 using the B from the paper \
Using the B from generate_B_i, when the noise level > 0.4 sometimes it does perform better other times it doesnt

In [94]:
# 2 qubit benchmarking circuits
for i in range(10):
    rho = random_circuit(2, 10)[0]
    
    noise_level = 0.01 * i
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

    true_value = execute(rho) # Fault tolerant quantum computer
    noisy_value = execute(noisy_rho) # Noisy quantum computer
    vd_value = vd(noisy_rho, 2, 1000)[0] # Noisy quantum computer + virtual distillation

    print(f"circuit: {i}, noise level: {noise_level}")
    print(f"True value: {true_value}")
    print(f"Noisy value: {noisy_value}")
    print(f"Virtual distillation value: {vd_value}")
    print()

circuit: 0, noise level: 0.0
True value: 1.0
Noisy value: 1.0
Virtual distillation value: 1.0

circuit: 1, noise level: 0.01
True value: 1.0
Noisy value: 0.2647159695625305
Virtual distillation value: 0.7484076433121019

circuit: 2, noise level: 0.02
True value: 1.0
Noisy value: 0.044080883264541626
Virtual distillation value: 0.2850877192982456

circuit: 3, noise level: 0.03
True value: 1.0
Noisy value: 0.019642934203147888
Virtual distillation value: 0.04924242424242424

circuit: 4, noise level: 0.04
True value: 1.0
Noisy value: 0.0011210441589355469
Virtual distillation value: 0.07037037037037037

circuit: 5, noise level: 0.05
True value: 1.0
Noisy value: 0.0009548664093017578
Virtual distillation value: -0.12222222222222222

circuit: 6, noise level: 0.06
True value: 1.0
Noisy value: 2.4646520614624023e-05
Virtual distillation value: -0.09701492537313433

circuit: 7, noise level: 0.07
True value: 1.0
Noisy value: 3.243982791900635e-05
Virtual distillation value: 0.1228813559322034

